## Homework: 使用完整的 YelpReviewFull 数据集训练，看 Acc 最高能到多少

In [1]:
# 准备数据集
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

In [2]:
# 预处理数据
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_datasets["train"]
# eval_dataset = tokenized_datasets["test"]
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

print("Training set:" + str(len(train_dataset)) + "  Test set:" + str(len(eval_dataset)))

c:\Users\gigifrog\.conda\envs\openaistudy\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Training set:650000  Test set:1000


In [3]:
# 加载BERT模型
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# 训练超参数
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-full-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir, 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [5]:
# 设定评估指标
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [6]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=100)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [7]:
trainer.train()

  0%|          | 0/121875 [00:00<?, ?it/s]

c:\Users\gigifrog\.conda\envs\openaistudy\lib\site-packages\transformers\utils\import_utils.py:526: FutureWarning: `is_compiling` is deprecated. Use `torch.compiler.is_compiling()` instead.
  return dynamo.is_compiling()


{'loss': 1.5464, 'learning_rate': 4.995897435897436e-05, 'epoch': 0.0}
{'loss': 1.2115, 'learning_rate': 4.991794871794872e-05, 'epoch': 0.0}
{'loss': 1.0936, 'learning_rate': 4.987692307692308e-05, 'epoch': 0.01}
{'loss': 1.0947, 'learning_rate': 4.983589743589744e-05, 'epoch': 0.01}


Checkpoint destination directory models/bert-base-cased-finetune-full-yelp\checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 1.0222, 'learning_rate': 4.979487179487179e-05, 'epoch': 0.01}


...

c:\Users\gigifrog\.conda\envs\openaistudy\lib\site-packages\transformers\utils\import_utils.py:526: FutureWarning: `is_compiling` is deprecated. Use `torch.compiler.is_compiling()` instead.
  return dynamo.is_compiling()


{'loss': 0.5635, 'learning_rate': 1.1282051282051281e-07, 'epoch': 2.99}
{'loss': 0.6047, 'learning_rate': 7.179487179487181e-08, 'epoch': 3.0}
{'loss': 0.5908, 'learning_rate': 3.076923076923077e-08, 'epoch': 3.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.7242664694786072, 'eval_accuracy': 0.7, 'eval_runtime': 9.0755, 'eval_samples_per_second': 110.186, 'eval_steps_per_second': 13.773, 'epoch': 3.0}
{'train_runtime': 48422.2171, 'train_samples_per_second': 40.271, 'train_steps_per_second': 2.517, 'train_loss': 0.7195786680563903, 'epoch': 3.0}


TrainOutput(global_step=121875, training_loss=0.7195786680563903, metrics={'train_runtime': 48422.2171, 'train_samples_per_second': 40.271, 'train_steps_per_second': 2.517, 'train_loss': 0.7195786680563903, 'epoch': 3.0})

In [8]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))
trainer.evaluate(small_test_dataset)

c:\Users\gigifrog\.conda\envs\openaistudy\lib\site-packages\transformers\utils\import_utils.py:526: FutureWarning: `is_compiling` is deprecated. Use `torch.compiler.is_compiling()` instead.
  return dynamo.is_compiling()


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.8781958222389221,
 'eval_accuracy': 0.61,
 'eval_runtime': 0.9969,
 'eval_samples_per_second': 100.312,
 'eval_steps_per_second': 13.041,
 'epoch': 3.0}

In [9]:
trainer.save_model(model_dir)
trainer.save_state()

In [10]:
trainer.evaluate(tokenized_datasets["test"])

c:\Users\gigifrog\.conda\envs\openaistudy\lib\site-packages\transformers\utils\import_utils.py:526: FutureWarning: `is_compiling` is deprecated. Use `torch.compiler.is_compiling()` instead.
  return dynamo.is_compiling()


  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.7412829399108887,
 'eval_accuracy': 0.69098,
 'eval_runtime': 442.0714,
 'eval_samples_per_second': 113.104,
 'eval_steps_per_second': 14.138,
 'epoch': 3.0}